# Code generator

Neural network that generates Golang code

In [7]:
import os

In [11]:
class Dataset(object):
    
    def __init__(self):
        dataset = self.load_dataset()
        self.encoded, self.vocab_to_int, self.int_to_vocab = dataset
        
    def load_dataset(self):
        if self.exists_clean_dataset():
            return self.load_clean_dataset()
        dataset = self.create_dataset()
        self.save_clean_dataset(dataset)
        return dataset

    def exists_clean_dataset(self):
        return os.path.isfile('dataset.dt')

    def load_clean_dataset(self):
        with open(Model.DATASET_FILE, "r") as d:
            dataset = json.loads(d.read())
            return (
                np.array(dataset["encoded"]),
                dataset["vocab_to_int"],
                dataset["int_to_vocab"]
            )
    
dataset = load_dataset()

NameError: name 'create_dataset' is not defined

In [1]:
class Model(object):
    pass    

In [4]:
class Trainer(Model):
    pass

In [5]:
class Predcitor(Model):
    pass